# 1. 사고심각도 예측 모델 구축

## 1.1. 실습 개요

이 노트북에서는 고속도로 교통사고 데이터를 기반으로 **사고심각도(사망 × 3 + 부상)** 를 예측하는 머신러닝 회귀 모델을 구축합니다.

단순히 모델을 학습시키는 것을 넘어서, **모델의 성능을 올바르게 해석하고 데이터 기반 의사결정 능력을 기르는 것**이 핵심 목표입니다.

## 1.2. 학습 목표

1. scikit-learn Pipeline을 활용한 전처리 + 모델 통합
2. ColumnTransformer와 OneHotEncoder를 활용한 범주형 변수 처리
3. 4가지 회귀 모델 비교 (Linear, RandomForest, XGBoost, LightGBM)
4. 평가 지표(MAE, RMSE, R²) 해석
5. Feature Importance 분석

## 1.3. 사용 모델

| 모델 | 목적 | 특징 |
|------|------|------|
| Linear Regression | Baseline | 단순 선형 관계 확인 |
| RandomForest | 안정적 예측 | 과적합에 강건, 해석 용이 |
| XGBoost | 고성능 | 비선형 패턴 학습 우수 |
| LightGBM | 고속·고성능 | 대규모 범주형 처리 강점 |

## 1.4. 평가 지표

| 지표 | 설명 | 해석 |
|------|------|------|
| MAE | 평균 절대 오차 | 낮을수록 좋음 |
| RMSE | 평균 제곱근 오차 | 큰 오차에 민감, 낮을수록 좋음 |
| R² | 결정 계수 | 1에 가까울수록 좋음, 음수면 baseline보다 나쁨 |

## 1.5. 중요 참고사항

이번 실습에서는 **사망·부상 컬럼을 모델 입력에서 제거**합니다.
- 이유: 사고심각도 = 사망 × 3 + 부상 이므로, 사망·부상을 포함하면 정보 누설(leakage) 발생
- 결과: 나머지 피처만으로는 예측이 어려워 성능이 낮게 나올 수 있음
- 의의: "타깃 관련 정보 누설을 제거하면 모델이 어떻게 영향받는지" 학습

In [ ]:
# ============================================
# 2. 패키지 설치 및 한글 폰트 설정
# ============================================

# --------------------------------------------
# 2.1. 추가 패키지 설치
# --------------------------------------------
# - XGBoost, LightGBM은 코랩에 기본 설치되어 있지 않을 수 있음
!pip install xgboost lightgbm -q

# --------------------------------------------
# 2.2. 한글 폰트 설정 (코랩 전용)
# --------------------------------------------
!apt-get update -qq
!apt-get install -y fonts-nanum

import matplotlib as mpl
import shutil

root = mpl.matplotlib_fname().replace("matplotlibrc", "")
target_font = root + "fonts/ttf/DejaVuSans.ttf"
nanum_font = "/usr/share/fonts/truetype/nanum/NanumGothic.ttf"
shutil.copyfile(nanum_font, target_font)

!rm -rf ~/.cache/matplotlib

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

# 한글 출력 테스트
plt.figure(figsize=(4, 3))
plt.title("한글 폰트 정상 출력 확인")
plt.xlabel("가로축")
plt.ylabel("세로축")
plt.plot([1, 2, 3], [1, 4, 2])
plt.show()

In [ ]:
# ============================================
# 3. 라이브러리 임포트 및 데이터 로드
# ============================================

import numpy as np
import pandas as pd
from math import sqrt

# scikit-learn 관련 모듈
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# 회귀 모델들
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

# 데이터 로드
file_path = "highway_accident_fe.csv"
df = pd.read_csv(file_path)

print("데이터 크기:", df.shape)
display(df.head())

In [ ]:
# ============================================
# 4. 타깃/특징 분리 및 피처 타입 분류
# ============================================

# --------------------------------------------
# 4.1. 타깃 변수 정의
# --------------------------------------------
target_col = "사고심각도"

# --------------------------------------------
# 4.2. 제외할 컬럼 정의
# --------------------------------------------
# - 사고심각도: 예측 타깃이므로 X에서 제외
# - 사망, 부상: 사고심각도 계산에 사용된 정보 (leakage 방지)
# - 사고일자: 고유값이 너무 많아 직접적 의미 없음
drop_cols = ["사고심각도", "사망", "부상", "사고일자"]
drop_cols = [c for c in drop_cols if c in df.columns]

# X: 입력 특징, y: 타깃
X = df.drop(columns=drop_cols)
y = df[target_col]

print("입력 특징 컬럼:", X.columns.tolist())

# --------------------------------------------
# 4.3. 범주형 / 수치형 컬럼 자동 분류
# --------------------------------------------
# - select_dtypes(): 데이터 타입에 따라 컬럼 선택
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()
numeric_cols = X.select_dtypes(include=["int64", "float64", "int32", "float32"]).columns.tolist()

print("\n범주형 컬럼:", categorical_cols)
print("수치형 컬럼:", numeric_cols)

In [ ]:
# ============================================
# 5. 전처리기(Preprocessor) 구성
# ============================================

# --------------------------------------------
# 5.1. OneHotEncoder 설정
# --------------------------------------------
# - 범주형 변수를 0/1 더미 변수로 변환
# - handle_unknown="ignore": 학습 시 없던 새로운 값이 나와도 에러 없이 처리
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

# --------------------------------------------
# 5.2. ColumnTransformer 설정
# --------------------------------------------
# - 서로 다른 컬럼 그룹에 서로 다른 전처리기 적용
# - cat: 범주형 컬럼 → OneHotEncoder
# - num: 수치형 컬럼 → 그대로 통과 (passthrough)
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categorical_cols),
        ("num", "passthrough", numeric_cols),
    ]
)

In [ ]:
# ============================================
# 6. 학습/테스트 데이터 분리
# ============================================

# - train_test_split(): 데이터를 학습용/테스트용으로 분리
# - test_size=0.2: 20%는 테스트, 80%는 학습
# - random_state=42: 재현성을 위한 난수 시드 고정
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("학습 데이터 크기:", X_train.shape)
print("테스트 데이터 크기:", X_test.shape)

In [ ]:
# ============================================
# 7. 회귀 모델 정의
# ============================================

models = {
    "RandomForest": RandomForestRegressor(
        n_estimators=300,      # 트리 개수
        random_state=42,
        n_jobs=-1,             # 모든 CPU 코어 사용
    ),
    "XGBoost": XGBRegressor(
        n_estimators=300,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1
    ),
    "LightGBM": LGBMRegressor(
        n_estimators=300,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42,
        n_jobs=-1
    ),
    "LinearRegression": LinearRegression()
}

# 성능 저장용 리스트
metrics_list = []

# Feature Importance 저장용 딕셔너리
feature_importances_dict = {}

In [ ]:
# ============================================
# 8. 모델 학습 및 평가
# ============================================

for model_name, model in models.items():
    print(f"\n{'=' * 50}")
    print(f"{model_name} 학습 중...")
    print("=" * 50)
    
    # --------------------------------------------
    # 8.1. Pipeline 구성
    # --------------------------------------------
    # - 전처리(preprocessor)와 모델(model)을 하나로 묶음
    # - fit() 호출 시 전처리 → 모델 학습이 순차 수행
    pipe = Pipeline([
        ("preprocessor", preprocessor),
        ("model", model),
    ])
    
    # --------------------------------------------
    # 8.2. 학습 및 예측
    # --------------------------------------------
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    
    # --------------------------------------------
    # 8.3. 평가 지표 계산
    # --------------------------------------------
    mae = mean_absolute_error(y_test, y_pred)
    rmse = sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    metrics_list.append({
        "model": model_name,
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2
    })
    
    print(f"MAE: {mae:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R²: {r2:.4f}")
    
    # --------------------------------------------
    # 8.4. Feature Importance 추출
    # --------------------------------------------
    # - get_feature_names_out(): 전처리 후 생성된 피처 이름 목록
    preproc = pipe.named_steps["preprocessor"]
    feature_names = preproc.get_feature_names_out()
    fitted_model = pipe.named_steps["model"]
    
    # 트리 모델: feature_importances_ 속성 사용
    # 선형 모델: coef_ 속성의 절대값 사용
    if hasattr(fitted_model, "feature_importances_"):
        importances = fitted_model.feature_importances_
    elif hasattr(fitted_model, "coef_"):
        importances = np.abs(fitted_model.coef_)
    else:
        importances = None
    
    if importances is not None:
        fi_df = pd.DataFrame({
            "feature": feature_names,
            "importance": importances
        }).sort_values("importance", ascending=False)
        
        feature_importances_dict[model_name] = fi_df
        print(f"\n[{model_name}] 상위 10개 Feature Importance:")
        print(fi_df.head(10).to_string(index=False))

In [ ]:
# ============================================
# 9. 모델 성능 비교
# ============================================

metrics_df = pd.DataFrame(metrics_list)
print("\n" + "=" * 50)
print("모델 성능 비교")
print("=" * 50)
display(metrics_df)

# RMSE 기준 정렬
metrics_df_sorted = metrics_df.sort_values("RMSE")
print("\n[RMSE 기준 정렬]")
display(metrics_df_sorted)

In [ ]:
# ============================================
# 10. 성능 시각화: RMSE 비교
# ============================================

plt.figure(figsize=(8, 4))
sns.barplot(data=metrics_df_sorted, x="model", y="RMSE", palette="Blues_d")
plt.title("모델별 RMSE 비교 (낮을수록 좋음)")
plt.xlabel("모델")
plt.ylabel("RMSE")
plt.tight_layout()
plt.show()

In [ ]:
# ============================================
# 11. 성능 시각화: R² 비교
# ============================================

plt.figure(figsize=(8, 4))
sns.barplot(
    data=metrics_df.sort_values("R2", ascending=False), 
    x="model", y="R2", palette="Reds_d"
)
plt.title("모델별 R² 비교 (1에 가까울수록 좋음)")
plt.xlabel("모델")
plt.ylabel("R²")
plt.axhline(y=0, color="gray", linestyle="--", linewidth=1)
plt.tight_layout()
plt.show()

# 12. 결과 해석

## 12.1. 성능 분석

모든 모델의 R²가 **음수**로 나타났습니다.

| 지표 | 의미 |
|------|------|
| R² < 0 | 모델이 평균값으로만 예측하는 baseline보다 못함 |
| R² = 0 | baseline과 동일한 성능 |
| R² = 1 | 완벽한 예측 |

## 12.2. 왜 이런 결과가 나왔는가?

**핵심 원인**: 사고심각도 = 사망 × 3 + 부상

이번 실습에서는 **사망·부상 컬럼을 입력에서 제외**했습니다.
즉, 타깃을 직접 설명하는 핵심 변수를 제거한 상태입니다.

남은 변수들(시간, 노선, 원인 등)은:
- 사고 발생 여부와는 관련이 있지만
- 사망/부상 발생 여부를 정확히 예측하기에는 **정보가 부족**합니다

## 12.3. 이 실습에서 배울 점

1. **데이터 누설(Leakage) 이해**: 타깃과 직접 관련된 변수를 포함하면 성능이 높아지지만, 실제 활용 시에는 의미가 없음
2. **모델 한계 인식**: 아무리 좋은 모델도 예측에 필요한 정보가 없으면 성능이 낮음
3. **R² 음수 해석**: 모델이 baseline보다 못하다는 의미로, 피처 선택/타깃 정의를 재검토해야 함

## 12.4. 개선 방향

다음 노트북에서는 **사고발생이정**을 타깃으로 예측합니다.
- 사고발생이정은 노선명, 방향 등과 구조적 관계가 있어 예측 가능성이 높음
- 이를 통해 "예측 가능한 타깃"과 "예측 어려운 타깃"의 차이를 체험